# Evaluation of $\langle \sigma v \rangle $ #

This is an interactive python demonstration that shows how the rate of a reaction varies with energy and temperature in the absence of resonances (i.e. assuming $S_0 = const.$) 

In [46]:
import numpy as np

# see https://www.astropy.org/ for documentation
import astropy.units as u 
import astropy.constants as c


%matplotlib notebook
from IPython.display import display
from ipywidgets import *
import matplotlib.pyplot as plt


def mass_numbers(z1=1,z2=2,n1=0,n2=0):
    a1,a2 = z1+n1,z2+n2
    return a1,a2


def reduced_A(a1,a2):
    a1,a2 = mass_numbers(a1,a2)
    A_r = (a1*a2)/(a1 + a2)
    return A_r 


def beta(z1 = 1,
         z2 = 2,
         n1 = 0,
         n2 = 0):  
    a1,a2 = mass_numbers(z1,z2,n1,n2)
    A_r = reduced_A(a1,a2)
    beta = 31.28*z1*z2*A_r**(0.5) * (u.keV)**0.5
    return beta 


def E0(z1 = 1,
       z2 = 1,
       n1 = 0,
       n2 = 0,
       T  = 1.5e6 * u.K):
    b = beta(z1,z2,n1,n2)
    E0 = (0.5*b*c.k_B*T)**(2./3.)
    return E0.to(u.keV) 

def Delta(z1 = 1,
          z2 = 1,
          n1 = 0,
          n2 = 0,
          T  = 1.5e6 * u.K):
    e0 = E0(z1,z2,n1,n2,T)
    Delta = 4/3**0.5 *(e0 * c.k_B * T)**0.5
    return Delta.to(u.keV)

    
def energy_range(z1 =1,
                 z2 = 1,
                 n1 = 0,
                 n2 = 0,
                 T = 1.5e6*u.K):
    e0 = E0(z1,z2,n1,n2,T)
    D = Delta(z1,z2,n1,n2,T)
    emin = np.max([0,(e0-4*D).value])
    emax = (e0 + 4*D).value
    energy = np.arange(emin,emax,(emax-emin)/1000.)
    return energy*u.keV
 
    
    
def SigmaV(z1=1, 
           z2=1,
           n1=0,
           n2=0,
           T  = 1.5e6,
           S0 = 1*u.keV*u.barn):  
    energy = energy_range(z1,z2,n1,n2,T)
    a1,a2 = mass_numbers(z1,z2,n1,n2)
    b = beta(z1,z2,n1,n2)
    A = reduced_A(a1,a2)
    mu = A*u.u
    f1 = (8/(mu*np.pi))**0.5
    f2 = S0/(c.k_B*T)**(3./2)
    f3 = np.exp(-energy/(T*c.k_B))
    f4 = np.exp(-b*energy**(-0.5))
    return (f1*f2*f3*f4).cgs

def plot_sv(T = 1e6*u.K,
           z1 = 1,
           z2 = 1,
           n1 = 0,
           n2 = 0,
           S0 = 1*u.keV*u.barn):
           energy = energy_range(z1,z2,n1,n2,T)
           sv = SigmaV(z1,z2,n1,n2,T,S0)
           plt.xlabel('Energy (kEV)')
           plt.plot(energy,sv,label=r'$\sigma v$')
           plt.legend()



@interact
def interactive_plot_sv(T = (1e6,1e10,1e6), #T = [1e6,5e6,1e7,5e7,1e8,5e8,1e9,5e9,1e10]*u.K,
           z1 = [1.,2.,3.,4.,5.],
           z2 = [1.,2.,3.,4.,5.],
           n1 = [0.,1.,2.,3.,4.,5.],
           n2 = [0.,1.,2.,3.,4.,5.],
           S0 = [1,10,100,1000]*u.keV*u.barn):
           plt.cla()
           energy = energy_range(z1,z2,n1,n2,T*u.K)
           sv = SigmaV(z1,z2,n1,n2,T*u.K,S0)
           I = np.trapz(sv,energy)
           
           logT_grid = np.arange(6,11,0.1)
           tg = 10**logT_grid 
           svg = np.array([SigmaV(1,1,0,0,temp*u.K).value.sum() for temp in tg])
            
           f = plt.figure(1)
           f.clear()

           plt.subplot(121)
           plt.xlabel('Energy (kEV)',size=18)
           plt.ylabel(r'$\sigma v$ normalized ',size=18)
           plt.plot(energy,sv/sv.max())
           plt.title(r"$\langle \sigma v \rangle=$ {:2E}".format(I),size=13)
           ax = f.add_subplot(122)
           ax.yaxis.tick_right()
           plt.subplot(122)
           plt.xlabel(r'$\log [Temperature (K)]$',size=18)
           plt.plot(logT_grid,np.log10(svg),label=r'$\log( \langle \sigma v \rangle)$',color='red')
           plt.title(r"Z$_1$={:};Z$_2$={:}".format(int(z1),int(z2)),size=13)
           plt.legend()
           f.canvas.draw()
    


interactive(children=(FloatSlider(value=5000000000.0, description='T', max=10000000000.0, min=1000000.0, step=…